In [4]:
# Import Libraries
import tweepy as tw
import configparser
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import re
import nltk
import datetime
from datetime import datetime as datetime_price
import pytz
from dateutil.relativedelta import relativedelta
import time

# Authentication for API

In [5]:
keys = pd.read_csv(r"C:\Users\Kutay\Desktop\My Docs\My Docs v2.0\Data Science\RA\Cryptocurrency\Input\TW API Keys\apikeys.csv")

my_api_key = keys["Keys"][0]
my_api_secret = keys["Keys"][1]

my_access_token = keys["Keys"][2]
my_access_token_secret = keys["Keys"][3]

In [6]:
# your Twitter API key ans access token
api_key = my_api_key
api_secret = my_api_secret

access_token = my_access_token
access_token_secret = my_access_token_secret

In [7]:
# authenticate
auth = tw.OAuthHandler(my_api_key, my_api_secret)
auth.set_access_token(access_token, access_token_secret)

In [8]:
api = tw.API(auth, wait_on_rate_limit=True)

# Insert the List of Users

In [9]:
user_df = pd.read_csv(r"C:\Users\Kutay\Desktop\My Docs\My Docs v2.0\Data Science\RA\Cryptocurrency\Input\Top 20 Cryptocurrencies and Related TW Accounts.csv", encoding= 'unicode_escape')
user_df

,Coin,ID,Owner/Influence,Name,Screen_name,Status,Why did I choose this person?
0,Ethereum,ethereum,Vitalik Buterin,vitalik.eth,VitalikButerin,co-founder,he is also known in crypto world
1,BNB,binancecoin,Changpeng Zhao,CZ ??Binance,cz_binance,CEO,NaN
2,XRP,ripple,Brad Garlinghouse,Brad Garlinghouse,bgarlinghouse,CEO,NaN
3,ADA(Cardano),cardano,Charles Hoskinson,Charles Hoskinson,IOHK_Charles,CEO/co-founder,NaN
4,DOGE,dogecoin,Elon Musk_doge,Elon Musk,elonmusk,influencer,"as a popular figure, highly interested in this..."
5,MATIC(Polygon),matic-network,Sandeep Nailwal,Sandeep | Polygon ????3??,sandeepnailwal,co-founder,he seems more active than other co-founders
6,DOT(Polkadot),polkadot,WEB3 Foundation,Web3 Foundation,Web3foundation,foundation,there is no account of CEO or sth
7,HEX,hex,Richard Heart,Richard Heart,RichardHeartWin,CEO,NaN
8,WTRX,wrapped-tron,Justin Sun,H.E. Justin Sun,justinsuntron,founder,same with TRX!
9,SHIB,shiba-inu,Elon Musk_shiba,Elon Musk,elonmusk,influencer,"as a popular figure, highly interested in this..."


# Taking Tweets of List of Users

user = "el33th4xor"
limit = 300

In [10]:
endDate = datetime.datetime.now()
startDate = endDate - relativedelta(days = 1) #days, months, years can be used to calculate

utc=pytz.UTC

startDate = utc.localize(startDate) 
endDate = utc.localize(endDate)
endDate

datetime.datetime(2023, 1, 16, 22, 0, 24, 6481, tzinfo=<UTC>)

In [24]:
#datetime format -> (year, month, day, hour, minute, second)
endDate = datetime.datetime(2022, 12, 31, 23, 59, 59)
startDate = datetime.datetime(2022, 12, 31, 0, 0, 0)
#endDate = datetime.datetime.now()
#startDate = endDate - relativedelta(days = 1) #days, months, years can be used to calculate

utc=pytz.UTC

startDate = utc.localize(startDate) 
endDate = utc.localize(endDate)

In [25]:
columns = ["Tweet_id", "Name", "Screen_name", "Tweet", "Date", "Category", "N_of_RT", "N_of_fav", "Related_coin"]

--old version of taking tweets
#cursor can be used for just one time, be careful about that, save and use use data frame/s for next steps
tweets = tw.Cursor(api.user_timeline, 
                   screen_name = user, count = 200, 
                   tweet_mode = "extended").items(limit)

In [26]:
def take_tweets(sc_name):
    tweets = []
    tmpTweets = api.user_timeline(screen_name = sc_name)
    for tweet in tmpTweets:
        if tweet.created_at < endDate and tweet.created_at > startDate:
            tweets.append(tweet)
        
    while (tmpTweets[-1].created_at > startDate):
        tmpTweets = api.user_timeline(screen_name = sc_name, max_id = tmpTweets[-1].id)
        for tweet in tmpTweets:
            if tweet.created_at < endDate and tweet.created_at > startDate:
                tweets.append(tweet)
    return tweets

In [27]:
def categorize_tweets(tweets):
    
    data = []
    term_rt = "RT"
    for tweet in tweets:
    
        #for consecutive tweets, next one is shown primaryly in data frame, I can check with using hours and minutes if necessary
        date = tweet.created_at
        date = date.strftime('%Y-%m-%d')
    
        #retweet and favorite counts
        number_of_retweet = tweet.retweet_count
        number_of_favorite = tweet.favorite_count
    
    
        #it's gonna be direct tweet if all the checks below fail
        category_specified = "direct"

        #retweet check
        if term_rt in tweet.text.split():

            category_specified = "retweet"

        #quote(rt with comment) check
        elif tweet.is_quote_status is True:
            category_specified = "quote"

        #reply check
        elif tweet.in_reply_to_status_id is not None:
            category_specified = "reply"

        data.append([tweet.id, tweet.user.name, tweet.user.screen_name, tweet.text, date, category_specified, number_of_retweet, number_of_favorite, ""])
    
    df_met = pd.DataFrame(data, columns = columns)
    return df_met

In [28]:
df = pd.DataFrame(columns = columns)

index_counter_1 = 0;
for x in user_df.Screen_name:
    
    taken_tweets = take_tweets(x)
    df_met_2 = categorize_tweets(taken_tweets)
    
    df_met_2["Related_coin"] = user_df["ID"][index_counter_1]
    index_counter_1 += 1
    
    df = df.append(df_met_2)

df = df[["Tweet_id", "Name", "Screen_name", "Related_coin", "Tweet", "Date", "Category", "N_of_RT", "N_of_fav"]]

aaaa
aaaa
bbb
bbb
bbb
bbb
bbb
bbb
bbb
bbb
bbb
bbb
aaaa
aaaa
bbb
bbb
bbb
bbb
bbb
bbb
aaaa
bbb
bbb
bbb
bbb
bbb
bbb
bbb
bbb
bbb
bbb
bbb
bbb
bbb
aaaa
bbb
bbb
bbb
bbb
bbb
bbb
bbb
bbb
bbb
bbb
bbb
bbb
bbb
bbb
bbb
bbb
bbb
bbb
bbb
bbb
aaaa
aaaa
bbb
bbb
bbb
bbb
bbb
bbb
bbb
aaaa
bbb
bbb
bbb
bbb
bbb
aaaa
bbb
bbb
bbb
bbb
bbb
bbb
bbb
bbb
bbb
bbb
bbb
bbb
bbb
aaaa
bbb
bbb
bbb
aaaa
bbb
bbb
bbb
bbb
bbb
aaaa
bbb
aaaa
aaaa
bbb
bbb
bbb
bbb
bbb
aaaa
aaaa
bbb
bbb
bbb


In [29]:
df

,Tweet_id,Name,Screen_name,Related_coin,Tweet,Date,Category,N_of_RT,N_of_fav
0,1609334004917964800,vitalik.eth,VitalikButerin,ethereum,@ChenWainuo @zengjiajun_eth @BlockGeekDima 我也祝...,2022-12-31,reply,3,11
1,1609312409491066882,vitalik.eth,VitalikButerin,ethereum,RT @VitalikButerin: @IAPonomarenko Happy new y...,2022-12-31,retweet,545,0
2,1609310102338543616,vitalik.eth,VitalikButerin,ethereum,"@IAPonomarenko Happy new year, and lots of lov...",2022-12-31,reply,545,5233
3,1609215293754163201,vitalik.eth,VitalikButerin,ethereum,@lexfridman @GretaThunberg @Cobratate I would ...,2022-12-31,reply,171,4967
4,1609209529706872833,vitalik.eth,VitalikButerin,ethereum,"@balajis Reaching the ""basic rollup scaling"" m...",2022-12-31,reply,398,2742
...,...,...,...,...,...,...,...,...,...
35,1609310410003324929,"Ethan Buchman (🐝,🦇)",buchmanster,cosmos,"4/ So 10k years ago, we were doing accounting,...",2022-12-31,reply,2,11
36,1609310410003324929,"Ethan Buchman (🐝,🦇)",buchmanster,cosmos,"4/ So 10k years ago, we were doing accounting,...",2022-12-31,reply,2,11
37,1609310408203976704,"Ethan Buchman (🐝,🦇)",buchmanster,cosmos,"3/ Since as long as we've had surpluses, we've...",2022-12-31,reply,1,8
38,1609310406505373698,"Ethan Buchman (🐝,🦇)",buchmanster,cosmos,"2/ Our story begins with farming, which takes ...",2022-12-31,reply,1,11


In [30]:
#removing duplications, reseting index
df.drop_duplicates(inplace = True)
df.reset_index(inplace = True, drop = True)

In [31]:
#seperating dat-month-year
df['Day'] = pd.DatetimeIndex(df['Date']).day
df['Month'] = pd.DatetimeIndex(df['Date']).month
df['Year'] = pd.DatetimeIndex(df['Date']).year

In [32]:
df.shape

(146, 12)

In [33]:
#null check --> if it is "False", we all ok
df.isna().any().any()

False

In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146 entries, 0 to 145
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Tweet_id      146 non-null    object
 1   Name          146 non-null    object
 2   Screen_name   146 non-null    object
 3   Related_coin  146 non-null    object
 4   Tweet         146 non-null    object
 5   Date          146 non-null    object
 6   Category      146 non-null    object
 7   N_of_RT       146 non-null    object
 8   N_of_fav      146 non-null    object
 9   Day           146 non-null    int64 
 10  Month         146 non-null    int64 
 11  Year          146 non-null    int64 
dtypes: int64(3), object(9)
memory usage: 13.8+ KB


In [35]:
#duplication check
df[df.duplicated(keep=False)].shape[0]

0

In [36]:
pd.set_option("max_rows", None)
df.to_excel(r"C:\Users\Kutay\Desktop\My Docs\My Docs v2.0\Data Science\RA\Cryptocurrency\RA_Cryptocurrency\Data\tweets.xlsx")
df.head()

,Tweet_id,Name,Screen_name,Related_coin,Tweet,Date,Category,N_of_RT,N_of_fav,Day,Month,Year
0,1609334004917964800,vitalik.eth,VitalikButerin,ethereum,@ChenWainuo @zengjiajun_eth @BlockGeekDima 我也祝...,2022-12-31,reply,3,11,31,12,2022
1,1609312409491066882,vitalik.eth,VitalikButerin,ethereum,RT @VitalikButerin: @IAPonomarenko Happy new y...,2022-12-31,retweet,545,0,31,12,2022
2,1609310102338543616,vitalik.eth,VitalikButerin,ethereum,"@IAPonomarenko Happy new year, and lots of lov...",2022-12-31,reply,545,5233,31,12,2022
3,1609215293754163201,vitalik.eth,VitalikButerin,ethereum,@lexfridman @GretaThunberg @Cobratate I would ...,2022-12-31,reply,171,4967,31,12,2022
4,1609209529706872833,vitalik.eth,VitalikButerin,ethereum,"@balajis Reaching the ""basic rollup scaling"" m...",2022-12-31,reply,398,2742,31,12,2022


In [37]:
#seperation according to categories 
retweet_df = df[df["Category"] == "retweet"]
retweet_df.reset_index(inplace = True, drop=True)
quote_df = df[df["Category"] == "quote"]
quote_df.reset_index(inplace = True, drop=True)
reply_df = df[df["Category"] == "reply"]
reply_df.reset_index(inplace = True, drop=True)
direct_df = df[df["Category"] == "direct"]
direct_df.reset_index(inplace = True, drop=True)
retweet_df.head()

,Tweet_id,Name,Screen_name,Related_coin,Tweet,Date,Category,N_of_RT,N_of_fav,Day,Month,Year
0,1609312409491066882,vitalik.eth,VitalikButerin,ethereum,RT @VitalikButerin: @IAPonomarenko Happy new y...,2022-12-31,retweet,545,0,31,12,2022
1,1609050268632772611,Charles Hoskinson,IOHK_Charles,cardano,RT @ikigai_myagi: @IOHK_Charles Took this pict...,2022-12-31,retweet,11,0,31,12,2022
2,1609207496434159621,Sandeep | Polygon 💜 Top 3 by impact,sandeepnailwal,matic-network,RT @eaglelex_eth: 11/The work at @0xPolygon 💜 ...,2022-12-31,retweet,3,0,31,12,2022
3,1609207483825889280,Sandeep | Polygon 💜 Top 3 by impact,sandeepnailwal,matic-network,RT @eaglelex_eth: 1/The future of #DeFi \n\nSo...,2022-12-31,retweet,13,0,31,12,2022
4,1609207102320377858,Sandeep | Polygon 💜 Top 3 by impact,sandeepnailwal,matic-network,RT @thematicman: 🎆 10 POLYGON NFT PROJECTS SET...,2022-12-31,retweet,65,0,31,12,2022


# Taking User Object

In [ ]:
user_try = api.get_user(screen_name = "el33th4xor")
user_try.name

# Follower - Followed

check this part of code later??????????????? cursor object is not itterable?

In [ ]:
user = "el33th4xor"
followers = []
get_followers = api.get_followers(screen_name = user)
#get_followers = tw.Cursor(api.get_followers, id = user)

for user in get_followers:
    followers.append(user)

In [ ]:
len(followers)

In [ ]:
columns_follower = ["User_id", "Name", "Screen_name", "Verification_st", "Followers_count", "Followed_count"]
data_followers = []

In [ ]:
for user in followers:
    data_followers.append([user.id, user.name, user.screen_name, user.verified, user.followers_count, user.friends_count])

In [ ]:
df_follower = pd.DataFrame(data_followers, columns = columns_follower)
df_follower

In [ ]:
df_follower.drop_duplicates(inplace = True)
df_follower.reset_index(inplace = True, drop = True)

# Statistics and Visualization

Graph should be addepted to new version of df(with lots of users)

In [ ]:
df.corr()

In [ ]:
plot1 = retweet_df.groupby("Day")["Tweet_id"].count()
plot2 = quote_df.groupby("Day")["Tweet_id"].count()
plot3 = reply_df.groupby("Day")["Tweet_id"].count()
plot4 = direct_df.groupby("Day")["Tweet_id"].count()
plt.figure(figsize = (14,7))
ax= plt.subplot()
ax.bar(plot1.index-0.3, plot1.values, width = 0.2, color = "pink", align = "center", label = "Retweet")
ax.bar(plot2.index-0.1, plot2.values, width = 0.2, color = "yellow", align = "center", label = "Quote")
ax.bar(plot3.index+0.1, plot3.values, width = 0.2, color = "purple", align = "center", label = "Reply")
ax.bar(plot4.index+0.3, plot4.values, width = 0.2, color = "grey", align = "center", label = "Direct")
plt.title("Number of Tweets According to Category per Day")
plt.ylabel("Number of Tweets")
plt.xlabel("Days")
#they are not sorted from 1 to 30 or 31, last they is today and first day is one month earlier. There is need for a solution also.
plt.xticks([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31]) #find a solution for changes of number of days in a month
plt.legend()
plt.show()